# NCLT Dataset

## Import

In [50]:
import os
from io import TextIOWrapper
import sys
import struct
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from pathlib import Path
from glob import glob
NCLT_ROOT = Path('/data/datasets/dataset_nclt')
NCLT_SEQS = [
    '2012-05-26', 
    '2012-08-20', 
    '2012-09-28',
    '2013-04-05']

## vel_sync

In [51]:
def getVelSyncPaths(seq_name):
    paths = glob(str(NCLT_ROOT / seq_name / 'velodyne_sync/*.bin'))
    paths.sort()
    return paths

In [52]:
scan_paths = getVelSyncPaths('2012-05-26')
print(len(scan_paths))

0


In [53]:
def convert(x_s, y_s, z_s):
    scaling = 0.005 # 5 mm
    offset = -100.0
    x = x_s * scaling + offset
    y = y_s * scaling + offset
    z = z_s * scaling + offset
    return x, y, z

def readPoint(f_bin: TextIOWrapper):
    buf = f_bin.read(2)
    if len(buf) != 2: return False, []
    x = struct.unpack('<H', buf)[0]
    y = struct.unpack('<H', f_bin.read(2))[0]
    z = struct.unpack('<H', f_bin.read(2))[0]
    i = struct.unpack('B', f_bin.read(1))[0]
    l = struct.unpack('B', f_bin.read(1))[0]
    x, y, z = convert(x, y, z)
    return True, [x, y, z, i]

def readSingleBin(bin_path: Path):
    print(bin_path)
    f_bin = open(bin_path, mode="rb") # encoding="ISO-8859-1"
    points = []
    
    while True:
        success, point = readPoint(f_bin)
        if success:
            points.append(point)
        else:
            break

    points = np.asarray(points)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], -points[:, 2], c=-points[:, 2], s=5, linewidths=0)
    plt.show()

In [ ]:
# readSingleBin(str(scan_paths[200]))

## Rearrange LiDAR Scans

In [56]:
## Rename
NCLT_ROOT = Path('/data/datasets/dataset_nclt')
NCLT_SEQS = [
    '2012-05-26', 
    '2012-08-20', 
    '2012-09-28',
    '2013-04-05']

def rearrage_lidar(root:Path, seq:str):
    lidar_data_fn = root / seq / 'lidar_data.txt'
    old_lidar_dir = root / seq / 'velodyne_sync'
    new_lidar_dir = root / seq / 'lidar'
    print("lidar_data_fn: ", lidar_data_fn)
    print("old_lidar_dir: ", old_lidar_dir)
    print("new_lidar_dir: ", new_lidar_dir)
    
    if lidar_data_fn.exists():
        print(f"Don't overwrite {lidar_data_fn}")
        return False
    if not root.exists():
        print(f"{root} doesn't exist!")
        return False
    if not old_lidar_dir.exists():
        print(f"{old_lidar_dir} doesn't exist!")
        return False
        
    os.makedirs(new_lidar_dir, exist_ok=True)
    f_data = open(lidar_data_fn, 'w')
    scan_paths = glob(f"{old_lidar_dir}/*.bin")
    scan_paths.sort()
    
    timestamps = []
    for scan_path in scan_paths:
        timestamp = float(Path(scan_path).name.split('.')[0]) / 1e6
        timestamps.append(timestamp)
        if len(timestamps) >= 2:
            if timestamps[-2] >= timestamps[-1]:
                print("Time order is wrong!")
                return False
    
    assert (len(timestamps) == len(scan_paths))
    
    for index in range(len(timestamps)):
        line = f"{timestamps[index]:.6f} lidar/{index:06d}.bin\n"
        old_scan_fn = scan_paths[index]
        new_scan_fn = f"{new_lidar_dir}/{index:06d}.bin"
        # print(old_scan_fn)
        # print(new_scan_fn)
        os.rename(old_scan_fn, new_scan_fn)
        f_data.write(line)

    f_data.close()
    return True

# success = rearrage_lidar(NCLT_ROOT, NCLT_SEQS[0])
# success = rearrage_lidar(NCLT_ROOT, NCLT_SEQS[1])
# success = rearrage_lidar(NCLT_ROOT, NCLT_SEQS[2])
# success = rearrage_lidar(NCLT_ROOT, NCLT_SEQS[3])

lidar_data_fn:  /data/datasets/dataset_nclt/2012-08-20/lidar_data.txt
old_lidar_dir:  /data/datasets/dataset_nclt/2012-08-20/velodyne_sync
new_lidar_dir:  /data/datasets/dataset_nclt/2012-08-20/lidar
lidar_data_fn:  /data/datasets/dataset_nclt/2012-09-28/lidar_data.txt
old_lidar_dir:  /data/datasets/dataset_nclt/2012-09-28/velodyne_sync
new_lidar_dir:  /data/datasets/dataset_nclt/2012-09-28/lidar
lidar_data_fn:  /data/datasets/dataset_nclt/2013-04-05/lidar_data.txt
old_lidar_dir:  /data/datasets/dataset_nclt/2013-04-05/velodyne_sync
new_lidar_dir:  /data/datasets/dataset_nclt/2013-04-05/lidar
